In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv("/Users/ohungchan/Downloads/all_data (1).csv")

# Drop rows with missing IDs
df = df.dropna(subset=["user_id"]).reset_index(drop=True)

# --- Grouping by user ---
# Each user is one group. We'll assign the majority label of their reviews
user_labels = df.groupby("user_id")["policy"].agg(lambda x: x.mode()[0])  # majority class
users = pd.DataFrame({"user_id": user_labels.index, "policy": user_labels.values})

# --- Train/val/test split at user level ---
train_users, temp_users = train_test_split(
    users,
    test_size=0.30,
    stratify=users["policy"],
    random_state=42
)

val_users, test_users = train_test_split(
    temp_users,
    test_size=2/3,  # 2/3 of 30% = 20%
    stratify=temp_users["policy"],
    random_state=42
)

# --- Map back to reviews ---
train_df = df[df["user_id"].isin(train_users["user_id"])]
val_df   = df[df["user_id"].isin(val_users["user_id"])]
test_df  = df[df["user_id"].isin(test_users["user_id"])]

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

Train: 13581 Val: 1964 Test: 3881


In [7]:
train_df.to_csv("train.csv", index=False, encoding='utf-8-sig')